In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn 

from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
X_train = pd.read_csv("./Dataset/CompressedData.csv")
X_train.head()

,Count,Open,High,Low,Close,Volume,VWAP,new_date,Asset_ID,Target
0,-0.420178,-0.231602,-0.231818,-0.231463,-0.231600,-0.163735,-0.231576,2018-01-01,0,0.000148
1,-0.406696,-0.231558,-0.231774,-0.231420,-0.231556,-0.163475,-0.231532,2018-01-02,0,0.000393
2,-0.412850,-0.231527,-0.231744,-0.231389,-0.231525,-0.163609,-0.231501,2018-01-03,0,0.000549
3,-0.411005,-0.231471,-0.231686,-0.231333,-0.231469,-0.163602,-0.231445,2018-01-04,0,0.000006
4,-0.389464,-0.230984,-0.231194,-0.230852,-0.230981,-0.162833,-0.230958,2018-01-05,0,0.005618


In [3]:
SEED_VALUE = 42
LEARNING_RATE = 0.01

y_train = X_train['Target']
X_train.drop(["Target", 'new_date', 'High', 'Low', 'Close', 'VWAP'], axis = 1, inplace=True)
X_train.head()

,Count,Open,Volume,Asset_ID
0,-0.420178,-0.231602,-0.163735,0
1,-0.406696,-0.231558,-0.163475,0
2,-0.412850,-0.231527,-0.163609,0
3,-0.411005,-0.231471,-0.163602,0
4,-0.389464,-0.230984,-0.162833,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size = 0.8, random_state = SEED_VALUE)
X_train.shape, X_test.shape

((14481, 4), (3621, 4))

In [5]:
# Fit basic XGBoost, lgbm and KNN and make predictions
model_xgb = XGBRegressor(random_state = 42, n_jobs = -1)
model_xgb.fit(X_train, y_train)
preds_xgb = model_xgb.predict(X_test)

model_lgb = LGBMRegressor()
model_lgb.fit(X_train, y_train)
preds_lgb = model_lgb.predict(X_test)

model_knn = KNeighborsRegressor(n_neighbors = 10)
model_knn.fit(X_train, y_train)
preds_knn = model_knn.predict(X_test)



print("XGB Base Error: ", 10e6*mse(preds_xgb, y_test))
print("LGB Base Error: ", 10e6*mse(preds_lgb, y_test))
print("KNN Base Error: ", 10e6*mse(preds_knn, y_test))

XGB Base Error:  2.443216458345216
LGB Base Error:  2.259477819615519
KNN Base Error:  2.3744811646447754


In [6]:
%%time

from sklearn.model_selection import cross_validate

def cross_validate_manual(X, y, model):
    cv = cross_validate(
        estimator = model,
        X = X,
        y = y,
        scoring = ["neg_mean_squared_error"],
        cv = 5,
    )
    
    return cv["test_neg_mean_squared_error"].mean()

CPU times: user 14 µs, sys: 2 µs, total: 16 µs
Wall time: 20.5 µs


In [7]:
def objective_LGB(trial):
    model_lgbm = LGBMRegressor()
    mean_lgbm = cross_validate_manual(X, y, model_lgbm)
    print("LGBM: ", np.abs(10e6*mean_lgbm))
    params = 
    return np.abs(10e6*mean_lgbm)